<a href="https://colab.research.google.com/github/Aani-122/my_learning/blob/main/first_deep_learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [3]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.3173216700000694
GPU (s):
0.08480238399999962
GPU speedup over CPU: 39x


In [5]:
import pandas as pd


In [6]:
df=pd.read_csv("https://raw.githubusercontent.com/FridahKimathi/Bank-Customer-Churn-Prediction/main/Data/churn.csv")

In [7]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [9]:
m=['RowNumber', 'CustomerId', 'Surname']
df.drop(m,inplace=True,axis=1)

In [10]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [11]:
df["Geography"].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [12]:
df["Gender"].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [13]:
df=pd.get_dummies(df,columns=['Geography',"Gender"],drop_first=True)

In [14]:
X=df.drop(["Exited"],axis=1)
y=df['Exited']

In [15]:
X.shape

(10000, 11)

In [16]:
y.shape

(10000,)

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [19]:
model=Sequential()
model.add(Dense(11,activation="sigmoid",input_dim=11))# 1 hidden layer
model.add(Dense(11,activation="sigmoid")) # 2 hidden layer
model.add(Dense(1,activation="sigmoid")) #output layer

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 11)                132       
                                                                 
 dense_1 (Dense)             (None, 11)                132       
                                                                 
 dense_2 (Dense)             (None, 1)                 12        
                                                                 
Total params: 276 (1.08 KB)
Trainable params: 276 (1.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [22]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [23]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [25]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [27]:
model.fit(X_train,y_train,epochs=50,verbose=1)

Epoch 1/50
250/250 [==============================] - 3s 3ms/step - loss: 0.5095 - accuracy: 0.7960
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4709 - accuracy: 0.7960
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4464 - accuracy: 0.7974
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4331 - accuracy: 0.8061
Epoch 5/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4275 - accuracy: 0.8115
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4245 - accuracy: 0.8099
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4220 - accuracy: 0.8104
Epoch 8/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4193 - accuracy: 0.8141
Epoch 9/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4162 - accuracy: 0.8173
Epoch 10/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4135 - accuracy: 0.8177

In [29]:
y_pred=model.predict(X_test)

63/63 [==============================] - 0s 4ms/step


parametre that are used in above line of code

1)epoch==>the cycle of each forward propogation and backword prapogation form 1 epoch

2)

```
# This is formatted as code
```



In [30]:
y_pred=y_pred.argmax(axis=-1)

In [31]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7975